In [ ]:
import tensorflow as tf 
from tensorflow import keras 
import matplotlib.pyplot as plt
import rasterio 
import raster_tools as rt 
import ai_tools as ai
import numpy as np
from joblib import dump

from keras.backend import clear_session
from numba import cuda

# Macro Feature Model

In [ ]:
input_dir = "training_data/macro_features/images"
target_dir =  "training_data/macro_features/annotations"
result_folder = "processed_raster"

In [ ]:
filters = 16
dropout= 0.15
batch_size= 12
num_val_samples=30
img_size=(512,512)
nbands=3
epochs = 250
nclasses=3

In [ ]:
train_generator, val_generator = ai.split_train_val(
    input_dir,
    target_dir,
    num_val_samples=num_val_samples,
    batch_size=batch_size,
    img_size=img_size,
    nbands=nbands, 
    nclasses=nclasses,
    augment=True
)

In [ ]:
input_img = keras.Input(img_size +(nbands,))
model = ai.get_multiUnet(
    input_img,
    nclasses=nclasses, 
    n_filters=filters, 
    dropout=dropout, 
    batchnorm=True
)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.categorical_crossentropy, 
    metrics="accuracy"
)

In [ ]:
history_micro = model.fit(
    train_generator,
    validation_data=val_generator, 
    batch_size=batch_size,
    epochs=epochs,
)

In [ ]:
dump(history_micro.history,"historys/history_macro_feature_training")
model.save("macro_feature_model")

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(8,4.5))

fig.suptitle("Model Performance - Macro-Features")

ax1.plot(history_micro['accuracy'])
ax1.plot(history_micro['val_accuracy'])

ax1.set_title('Accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')
ax1.legend(['training', 'validation'], loc='lower right')


ax1.set_yticks([i for i in np.arange(0,1.1,0.1)])
ax1.set_ylim(0,1)
ax2.set_ylim(0,14)
ax2.set_yticks([i for i in np.arange(0,15,1)])

ax2.plot(history_micro['loss'])
ax2.plot(history_micro['val_loss'])

ax2.set_title('Loss')
ax2.set_ylabel('Accuracy')
ax2.set_xlabel('Epoch')
ax2.legend(['training', 'validation'], loc='upper right')
plt.savefig("model_low_res.png",dpi=300)

# Prediction Macro Features

In [16]:
input_dir = "base_raster/"
ortho_fname = input_dir + "israelOrtho_20.tif"

In [17]:
rast_src,rast = rt.readTiff(ortho_fname)

In [ ]:
nbands, nrows, ncols = rast.shape
b,r,c = rast.shape 
r += r % img_size[0]
c += r % img_size[1]
pad_rast = np.zeros((b,r,c),np.float32)
pad_rast[:,:nrows,:ncols] = rast

In [ ]:
prediction_func = ai.predict(model)
result = rt.mem_padTileIterator(rast,prediction_func,img_size,nclasses)[:,:nrows,:ncols]

In [ ]:
# from skimage.filters import gaussian
# result_smooth = gaussian(result,1.5)

In [ ]:
profile = rast_src.profile 
profile.update(dtype=np.float32, count=nclasses)
with rasterio.open(result_folder + "macro_features_softmax.tif","w+",**profile) as dst:
    dst.write(result.astype(np.float32))


In [ ]:
profile = rast_src.profile 
profile.update(dtype=np.uint8, count=1)
with rasterio.open(result_folder + "macro_features_amax.tif","w+",**profile) as dst:
    dst.write(np.argmax(result,1).astype(np.uint8),1)


# Cleanup

In [ ]:

clear_session()
cuda.select_device(0)
cuda.close()